In [ ]:
import huggingface_hub
import datasets
import torchvision.transforms as transforms
import torch

#huggingface_hub.login()
imagenet_dataset = datasets.load_dataset("imagenet-1k",cache_dir='./data/imagenet')


In [1]:
import torch.nn as nn
import torch
class MaxtrixScaling(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.W = torch.nn.Parameter(torch.eye(num_classes))

    def forward(self, logits):
        return torch.matmul(logits, self.W)

a = MaxtrixScaling(3)
for i in a.parameters():
    print(i)
    

Parameter containing:
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]], requires_grad=True)


In [2]:
torch.nn.Parameter(torch.zeros(1))

Parameter containing:
tensor([0.], requires_grad=True)